# Bonds

In [1]:
import pandas as pd
import numpy as np

Below example follows SCHWESERNOTES™ 2019 LEVEL I CFA® BOOK 5: FIXED INCOME, DERIVATIVES, AND
ALTERNATIVE INVESTMENTS:

Consider a newly issued 10-year, \\$1,000 par value, 10% coupon, annual-pay bond. The
coupon payments will be $100 at the end of each year the \\$1,000 par value will be paid
at the end of year 10. What are the various present values for discount factors of 0.12, 0.10, and 0.08?

In [2]:
discount_rates = np.array([0.12, 0.10, 0.08])

In [3]:
pvs = np.pv(fv=1000, rate=discount_rates, nper=10,pmt=100)
pvs

array([ -886.99553943, -1000.        , -1134.20162798])

This shows us that as the discount rate **increases** the value, or price, of a bond **decreases**.

Calculating the value of the same bond with semmiannual payments:

In [4]:
pvs = np.pv(fv=1000, rate=discount_rates/2, nper=20,pmt=50)
pvs

array([ -885.30078781, -1000.        , -1135.90326345])

**zero-coupon bonds** have no coupons, therefore:

In [5]:
pvs = np.pv(fv=1000, rate=discount_rates, nper=10, pmt=0)
pvs

array([-321.97323659, -385.54328943, -463.19348808])

We can also use numpy to *find* the discount rate given a bond's price and par value.

In [6]:
np.rate(nper=10, pmt=100, pv=np.array([ -886.99553943, -1000, -1134.20162798]), fv=1000)

array([0.12, 0.1 , 0.08])

In reality the discount rate changes according to the market rate at different times, therefore:

In [7]:
ytm = np.rate(nper=3, pmt=50, pv=-1001.80, fv=1000)
print(f'YTM = {round(ytm*100, 2)}%')

YTM = 4.93%


Where **YTM** is the yield to maturity, the discount rate at which the sum of all future cash flows from the bond (coupons and principal) is equal to the current price of the bond. In other words, it is the internal rate of return if the investor holds the bond until maturity.

Source: https://www.investopedia.com/terms/y/yieldtomaturity.asp

In [8]:
np.pv(fv=1000, rate=0.015, nper=6,pmt=30)

-1085.4578074821281

# Loans

What is the monthly payment needed to pay off a \\$450,000 loan in 30 years at an annual interest rate of 4.0%?

In [10]:
np.pmt?

Signature: np.pmt(rate, nper, pv, fv=0, when='end')
Docstring:
Compute the payment against loan principal plus interest.

Given:
 * a present value, `pv` (e.g., an amount borrowed)
 * a future value, `fv` (e.g., 0)
 * an interest `rate` compounded once per period, of which
   there are
 * `nper` total
 * and (optional) specification of whether payment is made
   at the beginning (`when` = {'begin', 1}) or the end
   (`when` = {'end', 0}) of each period

Return:
   the (fixed) periodic payment.

Parameters
----------
rate : array_like
    Rate of interest (per period)
nper : array_like
    Number of compounding periods
pv : array_like
    Present value
fv : array_like,  optional
    Future value (default = 0)
when : {{'begin', 1}, {'end', 0}}, {string, int}
    When payments are due ('begin' (1) or 'end' (0))

Returns
-------
out : ndarray
    Payment against loan plus interest.  If all input is scalar, returns a
    scalar float.  If any input is array_like, returns payment for each
  

In [11]:
pmt = np.pmt(rate=0.04/12, nper=12*30, pv=450000, fv=0)
print(f'PMT = ${round(abs(pmt), 2)}')

PMT = $2148.37


The principal portion of the payment for a given period becomes:

In [12]:
ppmt = np.ppmt(rate=0.04/12, per=1, nper=12*30, pv=450000, fv=0)
print(f'PPMT = ${round(abs(ppmt), 2)}')

PPMT = $648.37


And the interest portion of the payment for a given period becomes:

In [13]:
ipmt = np.ipmt(rate=0.04/12, per=1, nper=12*30, pv=450000, fv=0)
print(f'PPMT = ${round(abs(ipmt), 2)}')

PPMT = $1500.0


We can create an amortization schedule by using a pandas dataframe and lessons from the Data Analysis notebooks:

In [14]:
amortization = pd.DataFrame({'period':np.arange(12*30)+1})

In [15]:
amortization.head()

,period
0,1
1,2
2,3
3,4
4,5


In [16]:
amortization['principal'] = np.abs(np.ppmt(rate=0.04/12, per=amortization.period, nper=12*30, pv=450000, fv=0))

In [17]:
amortization['interest'] = np.abs(np.ipmt(rate=0.04/12, per=amortization.period, nper=12*30, pv=450000, fv=0))

In [18]:
amortization['payment'] = amortization['principal'] + amortization['interest'] 

In [19]:
amortization['balance'] = 450000 - amortization['principal'].cumsum()

In [20]:
pd.options.display.float_format = '{:,.2f}'.format

In [21]:
amortization

,period,principal,interest,payment,balance
0,1,648.37,"1,500.00","2,148.37","449,351.63"
1,2,650.53,"1,497.84","2,148.37","448,701.10"
2,3,652.70,"1,495.67","2,148.37","448,048.40"
3,4,654.87,"1,493.49","2,148.37","447,393.53"
4,5,657.06,"1,491.31","2,148.37","446,736.47"
...,...,...,...,...,...
355,356,"2,112.92",35.45,"2,148.37","8,522.34"
356,357,"2,119.96",28.41,"2,148.37","6,402.38"
357,358,"2,127.03",21.34,"2,148.37","4,275.35"
358,359,"2,134.12",14.25,"2,148.37","2,141.23"
